# Plotting Module User Guide

This notebook provides a comprehensive guide to using the `plots` module from the `algoshort` package for financial data visualization.

## Table of Contents

1. [Setup and Installation](#1-setup-and-installation)
2. [Understanding the Plots Module](#2-understanding-the-plots-module)
3. [Data Preparation](#3-data-preparation)
4. [Price Comparison Plots](#4-price-comparison-plots)
5. [Signal Visualization](#5-signal-visualization)
6. [Regime Detection Plots](#6-regime-detection-plots)
7. [Performance & P&L Plots](#7-performance--pl-plots)
8. [Position Sizing Visualization](#8-position-sizing-visualization)
9. [Customization and Integration](#9-customization-and-integration)
10. [Best Practices and Tips](#10-best-practices-and-tips)

## 1. Setup and Installation

First, let's import the required modules and configure the plotting environment.

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging

# Configure logging to see debug messages
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Configure matplotlib for better notebook display
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# Import plotting modules
from algoshort.plots import (
    # Price comparison
    plot_abs_rel,
    
    # Signal plots
    plot_signal_bo,
    plot_signal_tt,
    plot_signal_ma,
    plot_signal_abs,
    plot_signal_rel,
    
    # Regime plots
    plot_regime_abs,
    plot_regime_rel,
    
    # Performance plots
    plot_profit_loss,
    plot_price_signal_cumreturns,
    plot_equity_risk,
    
    # Position sizing plots
    plot_shares_signal,
    plot_equity_amount,
    
    # Constants
    DEFAULT_FIG_WIDTH,
    DEFAULT_FIG_HEIGHT,
    DATE_COLUMN,
    CLOSE_COLUMN,
    RCLOSE_COLUMN,
)

print("Imports successful!")
print(f"\nModule Constants:")
print(f"  DEFAULT_FIG_WIDTH: {DEFAULT_FIG_WIDTH}")
print(f"  DEFAULT_FIG_HEIGHT: {DEFAULT_FIG_HEIGHT}")
print(f"  DATE_COLUMN: '{DATE_COLUMN}'")
print(f"  CLOSE_COLUMN: '{CLOSE_COLUMN}'")
print(f"  RCLOSE_COLUMN: '{RCLOSE_COLUMN}'")

## 2. Understanding the Plots Module

### Available Functions

The `plots` module provides 13 visualization functions organized by category:

| Category | Function | Description |
|----------|----------|-------------|
| **Price Comparison** | `plot_abs_rel` | Absolute vs relative price comparison |
| **Signals** | `plot_signal_bo` | Breakout signal with high/low channels |
| | `plot_signal_tt` | Turtle trading signal |
| | `plot_signal_ma` | Moving average (SMA/EMA) signals |
| | `plot_signal_abs` | Absolute price with floor/ceiling |
| | `plot_signal_rel` | Relative price with floor/ceiling |
| **Regimes** | `plot_regime_abs` | Absolute regime detection |
| | `plot_regime_rel` | Relative regime detection |
| **Performance** | `plot_profit_loss` | Cumulative P&L and daily changes |
| | `plot_price_signal_cumreturns` | Price, signal, and cumulative returns |
| | `plot_equity_risk` | Equity with drawdown and risk levels |
| **Position Sizing** | `plot_shares_signal` | Share quantities by sizing method |
| | `plot_equity_amount` | Equity curves for different strategies |

### Common Function Signature

All plot functions follow a consistent pattern:

```python
def plot_function(
    df: pd.DataFrame,        # Data with required columns
    ticker: str,             # Ticker symbol for title
    **kwargs                 # Function-specific parameters
) -> Tuple[plt.Figure, plt.Axes]:
    """Returns (Figure, Axes) tuple for further customization."""
```

### Input Validation

All functions validate:
- DataFrame is not empty
- Required columns exist
- Ticker is not None or empty
- Numeric parameters are positive integers (where applicable)

### Memory Management

All functions call `plt.close()` after `plt.show()` to prevent memory leaks when generating many plots.

## 3. Data Preparation

Let's create sample data for our demonstrations. In production, you would load this from the `YFinanceDataHandler` or other data sources.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate 2 years of daily trading data
n_days = 504  # ~2 years of trading days
dates = pd.date_range('2022-01-03', periods=n_days, freq='B')

# Generate realistic price movements
returns = np.random.normal(0.0005, 0.018, n_days)  # Daily returns
close = 150 * np.exp(np.cumsum(returns))  # Cumulative returns

# Generate OHLC from close
high = close * (1 + np.abs(np.random.normal(0, 0.008, n_days)))
low = close * (1 - np.abs(np.random.normal(0, 0.008, n_days)))
open_price = low + (high - low) * np.random.uniform(0.2, 0.8, n_days)

# Generate benchmark (SPY-like)
bm_returns = np.random.normal(0.0004, 0.012, n_days)
rclose = 100 * np.exp(np.cumsum(bm_returns))

# Create base DataFrame
df = pd.DataFrame({
    'date': dates,
    'open': open_price,
    'high': high,
    'low': low,
    'close': close,
    'volume': np.random.randint(5_000_000, 20_000_000, n_days),
    'rclose': rclose,  # Relative close (rebased to benchmark)
})

print(f"Sample data created:")
print(f"  Shape: {df.shape}")
print(f"  Date range: {df['date'].iloc[0].strftime('%Y-%m-%d')} to {df['date'].iloc[-1].strftime('%Y-%m-%d')}")
print(f"  Close price range: ${df['close'].min():.2f} - ${df['close'].max():.2f}")
print(f"\nColumns: {list(df.columns)}")
df.head()

In [ ]:
# Add breakout signal columns
window = 20

# Rolling high/low channels
df[f'hi_{window}'] = df['close'].rolling(window).max()
df[f'lo_{window}'] = df['close'].rolling(window).min()

# Breakout signal: 1 = breakout up, -1 = breakout down, 0 = no breakout
df[f'bo_{window}'] = np.where(
    df['close'] > df[f'hi_{window}'].shift(1), 1,
    np.where(df['close'] < df[f'lo_{window}'].shift(1), -1, 0)
)

# Also add relative breakout columns
df[f'rhi_{window}'] = df['rclose'].rolling(window).max()
df[f'rlo_{window}'] = df['rclose'].rolling(window).min()
df[f'rbo_{window}'] = np.where(
    df['rclose'] > df[f'rhi_{window}'].shift(1), 1,
    np.where(df['rclose'] < df[f'rlo_{window}'].shift(1), -1, 0)
)

print(f"Breakout columns added for window={window}")
print(f"  Absolute: hi_{window}, lo_{window}, bo_{window}")
print(f"  Relative: rhi_{window}, rlo_{window}, rbo_{window}")

In [ ]:
# Add turtle trading signal columns
fast = 20
slow = 55

# Fast and slow channel breakouts
hi_fast = df['close'].rolling(fast).max()
lo_fast = df['close'].rolling(fast).min()
hi_slow = df['close'].rolling(slow).max()
lo_slow = df['close'].rolling(slow).min()

# Turtle signal: Enter on slow breakout, exit on fast breakout
# Simplified version
df[f'turtle_{slow}{fast}'] = np.where(
    df['close'] > hi_slow.shift(1), 1,
    np.where(df['close'] < lo_slow.shift(1), -1, 0)
)

print(f"Turtle trading column added: turtle_{slow}{fast}")

In [ ]:
# Add moving average signal columns
st, mt, lt = 10, 20, 50  # Short, medium, long term

# Calculate SMAs
sma_st = df['close'].rolling(st).mean()
sma_mt = df['close'].rolling(mt).mean()
sma_lt = df['close'].rolling(lt).mean()

# Calculate EMAs
ema_st = df['close'].ewm(span=st).mean()
ema_mt = df['close'].ewm(span=mt).mean()
ema_lt = df['close'].ewm(span=lt).mean()

# MA signals: 1 = bullish alignment, -1 = bearish, 0 = mixed
df[f'sma_{st}{mt}{lt}'] = np.where(
    (sma_st > sma_mt) & (sma_mt > sma_lt), 1,
    np.where((sma_st < sma_mt) & (sma_mt < sma_lt), -1, 0)
)

df[f'ema_{st}{mt}{lt}'] = np.where(
    (ema_st > ema_mt) & (ema_mt > ema_lt), 1,
    np.where((ema_st < ema_mt) & (ema_mt < ema_lt), -1, 0)
)

print(f"Moving average columns added:")
print(f"  sma_{st}{mt}{lt}, ema_{st}{mt}{lt}")

In [ ]:
# Add floor/ceiling signal columns (absolute)
# These would normally come from regime detection

# Swing highs and lows (simplified)
df['hi3'] = df['close'].rolling(5, center=True).max()
df['lo3'] = df['close'].rolling(5, center=True).min()

# Ceiling and floor points (where price equals swing high/low)
df['clg'] = np.where(
    (df['close'] == df['hi3']) & (df['close'].shift(1) < df['close']),
    df['close'], np.nan
)
df['flr'] = np.where(
    (df['close'] == df['lo3']) & (df['close'].shift(1) > df['close']),
    df['close'], np.nan
)

# Regime change and regime
df['rg_ch'] = np.where(
    df['clg'].notna(), -1,
    np.where(df['flr'].notna(), 1, 0)
)
df['rg'] = df['rg_ch'].cumsum()

# Relative versions
df['rh3'] = df['rclose'].rolling(5, center=True).max()
df['rl3'] = df['rclose'].rolling(5, center=True).min()
df['rclg'] = np.where(
    (df['rclose'] == df['rh3']) & (df['rclose'].shift(1) < df['rclose']),
    df['rclose'], np.nan
)
df['rflr'] = np.where(
    (df['rclose'] == df['rl3']) & (df['rclose'].shift(1) > df['rclose']),
    df['rclose'], np.nan
)
df['rrg_ch'] = np.where(
    df['rclg'].notna(), -1,
    np.where(df['rflr'].notna(), 1, 0)
)
df['rrg'] = df['rrg_ch'].cumsum()

print("Floor/ceiling columns added:")
print("  Absolute: hi3, lo3, clg, flr, rg_ch, rg")
print("  Relative: rh3, rl3, rclg, rflr, rrg_ch, rrg")

In [ ]:
# Add P&L and performance columns
signal = df[f'turtle_{slow}{fast}']
daily_returns = df['close'].pct_change()

# Strategy returns (signal from previous day * today's return)
strategy_returns = signal.shift(1) * daily_returns

# Cumulative returns
df['tt_cumul'] = (1 + strategy_returns).cumprod() - 1

# P&L columns
initial_capital = 100_000
df['tt_chg1D'] = strategy_returns * initial_capital
df['tt_PL_cum'] = df['tt_chg1D'].cumsum()

# Stop loss (simplified - 5% trailing)
df['stop_loss'] = df['close'].rolling(20).min() * 0.95

print("P&L columns added:")
print("  tt_cumul, tt_chg1D, tt_PL_cum, stop_loss")

In [ ]:
# Add equity and risk columns
equity = initial_capital + df['tt_PL_cum'].fillna(0)
df['peak_eqty'] = equity.expanding().max()
df['drawdown'] = (equity - df['peak_eqty']) / df['peak_eqty']
df['tolerance'] = df['peak_eqty'] * 0.9  # 10% max drawdown tolerance

# Risk levels for different position sizing methods
df['constant_risk'] = 0.02  # 2% constant risk
df['convex_risk'] = 0.02 + np.abs(df['drawdown']) * 0.5  # Increases with drawdown
df['concave_risk'] = np.maximum(0.005, 0.02 - np.abs(df['drawdown']) * 0.5)  # Decreases with drawdown

print("Equity/risk columns added:")
print("  peak_eqty, drawdown, tolerance")
print("  constant_risk, convex_risk, concave_risk")

In [ ]:
# Add position sizing columns
capital = 100_000
price = df['close']

# Different position sizing methods
df['shs_eql'] = (capital / price).astype(int)  # Equal dollar allocation
df['shs_fxd'] = 100  # Fixed 100 shares
df['shs_ccv'] = (capital * df['concave_risk'] / price).astype(int)  # Concave sizing
df['shs_cvx'] = (capital * df['convex_risk'] / price).astype(int)  # Convex sizing

# Equity curves for different methods
df['equal_weight'] = capital + (df['shs_eql'].shift(1) * df['close'].diff()).cumsum().fillna(0)
df['constant'] = capital + (df['shs_fxd'] * df['close'].diff()).cumsum().fillna(0)
df['concave'] = capital + (df['shs_ccv'].shift(1) * df['close'].diff()).cumsum().fillna(0)
df['convex'] = capital + (df['shs_cvx'].shift(1) * df['close'].diff()).cumsum().fillna(0)
df['tt_PL_cum_fx'] = capital + df['tt_PL_cum'].fillna(0)

# Signal column for shares plot
df['signal'] = df[f'turtle_{slow}{fast}']

print("Position sizing columns added:")
print("  Shares: shs_eql, shs_fxd, shs_ccv, shs_cvx")
print("  Equity: equal_weight, constant, concave, convex, tt_PL_cum_fx")

In [ ]:
# Summary of all columns
print(f"Final DataFrame shape: {df.shape}")
print(f"\nAll columns ({len(df.columns)}):")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

## 4. Price Comparison Plots

### plot_abs_rel()

Compare absolute price with relative price (rebased to benchmark).

**Required columns:** `date`, `close`, `rclose`

In [ ]:
# Basic absolute vs relative plot
fig, ax = plot_abs_rel(df, 'AAPL', 'SPY')

# The function returns (Figure, Axes) for further customization
print(f"\nReturned: Figure={type(fig).__name__}, Axes={type(ax).__name__}")

In [ ]:
# Error handling demonstration
print("Error handling examples:\n")

# 1. Empty DataFrame
try:
    plot_abs_rel(pd.DataFrame(), 'AAPL', 'SPY')
except ValueError as e:
    print(f"1. Empty DataFrame: {e}")

# 2. None ticker
try:
    plot_abs_rel(df, None, 'SPY')
except ValueError as e:
    print(f"2. None ticker: {e}")

# 3. Missing columns
try:
    df_missing = df[['date', 'close']].copy()
    plot_abs_rel(df_missing, 'AAPL', 'SPY')
except ValueError as e:
    print(f"3. Missing columns: {e}")

## 5. Signal Visualization

### plot_signal_bo() - Breakout Signals

Visualize price with high/low channels and breakout signals.

**Required columns:** `close`, `hi_{window}`, `lo_{window}`, `bo_{window}`

In [ ]:
# Breakout signal plot (absolute)
fig, ax = plot_signal_bo(df, window=20, ticker='AAPL', relative=False)

In [ ]:
# Breakout signal plot (relative)
fig, ax = plot_signal_bo(df, window=20, ticker='AAPL', relative=True)

### plot_signal_tt() - Turtle Trading Signals

Visualize turtle trading dual-channel breakout signals.

**Required columns:** `close`, `turtle_{slow}{fast}`

In [ ]:
# Turtle trading signal plot
fig, ax = plot_signal_tt(df, fast=20, slow=55, ticker='AAPL')

### plot_signal_ma() - Moving Average Signals

Visualize SMA and EMA crossover signals.

**Required columns:** `close`, `sma_{st}{mt}{lt}`, `ema_{st}{mt}{lt}`

In [ ]:
# Moving average signal plot (creates 2 subplots: SMA and EMA)
fig, (ax1, ax2) = plot_signal_ma(df, st=10, mt=20, lt=50, ticker='AAPL')

### plot_signal_abs() / plot_signal_rel() - Floor/Ceiling Signals

Visualize price with swing highs/lows and floor/ceiling markers.

**Required columns (abs):** `close`, `hi3`, `lo3`, `clg`, `flr`, `rg_ch`, `rg`  
**Required columns (rel):** `rclose`, `rh3`, `rl3`, `rclg`, `rflr`, `rrg_ch`, `rrg`

In [ ]:
# Absolute signal plot
fig, ax = plot_signal_abs(df, 'AAPL')

In [ ]:
# Relative signal plot
fig, ax = plot_signal_rel(df, 'AAPL')

## 6. Regime Detection Plots

### plot_regime_abs() / plot_regime_rel()

Visualize regime states with floor/ceiling markers.

**Required columns (abs):** `close`, `rg`, `lo3`, `hi3`, `clg`, `flr`, `rg_ch`  
**Required columns (rel):** `rclose`, `rrg`, `rl3`, `rh3`, `rclg`, `rflr`, `rrg_ch`

In [ ]:
# Absolute regime plot
fig, ax = plot_regime_abs(df, 'AAPL')

In [ ]:
# Relative regime plot
fig, ax = plot_regime_rel(df, 'AAPL')

## 7. Performance & P&L Plots

### plot_profit_loss()

Visualize cumulative P&L and daily changes.

**Required columns:** `tt_PL_cum`, `tt_chg1D`

In [ ]:
# P&L plot
fig, ax = plot_profit_loss(df, 'AAPL', method='Turtle')

### plot_price_signal_cumreturns()

Visualize price, signal, stop-loss, and cumulative returns together.

**Required columns:** `close`, `stop_loss`, `{signal}`, `tt_cumul`

In [ ]:
# Price, signal, and cumulative returns
fig, ax = plot_price_signal_cumreturns(df, 'AAPL', signal='signal', method='Turtle')

### plot_equity_risk()

Visualize equity curve with drawdown and risk levels.

**Required columns:** `close`, `peak_eqty`, `tolerance`, `drawdown`, `constant_risk`, `convex_risk`, `concave_risk`

In [ ]:
# Equity risk plot (creates 2 subplots: drawdown and risk levels)
fig, (ax1, ax2) = plot_equity_risk(df, 'AAPL', method='Turtle')

## 8. Position Sizing Visualization

### plot_shares_signal()

Visualize share quantities for different position sizing methods.

**Required columns:** `shs_eql`, `shs_fxd`, `shs_ccv`, `shs_cvx`, `{signal}`

In [ ]:
# Shares by position sizing method
fig, ax = plot_shares_signal(df, 'AAPL', signal='signal', method='Turtle')

### plot_equity_amount()

Visualize equity curves for different position sizing strategies.

**Required columns:** `constant`, `concave`, `convex`, `equal_weight`, `tt_PL_cum_fx`

In [ ]:
# Equity curves comparison
fig, ax = plot_equity_amount(df, 'AAPL', method='Turtle')

## 9. Customization and Integration

### Using Returned Figure/Axes

All plot functions return `(Figure, Axes)` tuples, allowing further customization.

In [ ]:
# Note: Since plt.show() and plt.close() are called inside the functions,
# direct customization after calling is limited.
# For full customization, you would modify the source or use matplotlib directly.

# Here's how you might create a custom version:
def custom_abs_rel_plot(df, ticker, benchmark):
    """Custom version of plot_abs_rel with more control."""
    # Validate and prepare data
    if df.empty:
        raise ValueError("DataFrame cannot be empty")
    
    plot_df = df.copy()
    if 'date' in plot_df.columns:
        plot_df = plot_df.set_index('date')
    
    # Create figure with custom styling
    fig, ax = plt.subplots(figsize=(16, 8))
    
    # Plot with custom colors
    ax.plot(plot_df.index, plot_df['close'], 'b-', linewidth=1.5, label=f'{ticker} (Absolute)', alpha=0.8)
    ax.plot(plot_df.index, plot_df['rclose'], 'r--', linewidth=1.5, label=f'{ticker} vs {benchmark} (Relative)', alpha=0.8)
    
    # Customize
    ax.set_title(f'{ticker}: Absolute vs Relative to {benchmark}', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Price', fontsize=12)
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    
    # Add annotations
    ax.annotate(f'Final: ${plot_df["close"].iloc[-1]:.2f}', 
                xy=(plot_df.index[-1], plot_df['close'].iloc[-1]),
                xytext=(10, 10), textcoords='offset points',
                fontsize=10, color='blue')
    
    plt.tight_layout()
    plt.show()
    return fig, ax

# Use custom version
fig, ax = custom_abs_rel_plot(df, 'AAPL', 'SPY')

### Integration with Other Modules

The plots module integrates with other `algoshort` modules in a typical workflow:

In [ ]:
# Example workflow (conceptual)
print("""
TYPICAL WORKFLOW
================

# 1. Load data
from algoshort.yfinance_handler import YFinanceDataHandler
handler = YFinanceDataHandler()
df = handler.get_ohlc_data('AAPL')

# 2. Calculate relative prices
from algoshort.ohlcprocessor import OHLCProcessor
processor = OHLCProcessor(df, benchmark='SPY')
df = processor.calculate_relative_prices()

# 3. Detect regimes
from algoshort.regimes import RegimeDetector
detector = RegimeDetector(df)
df = detector.compute('floor_ceiling')

# 4. Calculate signals
from algoshort.signals import turtle_trader
df = turtle_trader(df, slow=55, fast=20)

# 5. Calculate returns
from algoshort.returns import ReturnsCalculator
returns_calc = ReturnsCalculator(df)
df = returns_calc.calculate_returns(signal_columns=['turtle_5520'])

# 6. Visualize!
from algoshort.plots import (
    plot_abs_rel,
    plot_regime_abs,
    plot_price_signal_cumreturns,
    plot_equity_amount
)

plot_abs_rel(df, 'AAPL', 'SPY')
plot_regime_abs(df, 'AAPL')
plot_price_signal_cumreturns(df, 'AAPL', 'turtle_5520', 'Turtle')
plot_equity_amount(df, 'AAPL', 'Turtle')
""")

## 10. Best Practices and Tips

### Data Preparation Checklist

| Function | Required Columns |
|----------|------------------|
| `plot_abs_rel` | date, close, rclose |
| `plot_signal_bo` | close, hi_N, lo_N, bo_N |
| `plot_signal_tt` | close, turtle_XY |
| `plot_signal_ma` | close, sma_XYZ, ema_XYZ |
| `plot_signal_abs` | close, hi3, lo3, clg, flr, rg_ch, rg |
| `plot_signal_rel` | rclose, rh3, rl3, rclg, rflr, rrg_ch, rrg |
| `plot_regime_abs` | close, rg, lo3, hi3, clg, flr, rg_ch |
| `plot_regime_rel` | rclose, rrg, rl3, rh3, rclg, rflr, rrg_ch |
| `plot_profit_loss` | tt_PL_cum, tt_chg1D |
| `plot_price_signal_cumreturns` | close, stop_loss, {signal}, tt_cumul |
| `plot_equity_risk` | close, peak_eqty, tolerance, drawdown, *_risk |
| `plot_shares_signal` | shs_eql, shs_fxd, shs_ccv, shs_cvx, {signal} |
| `plot_equity_amount` | constant, concave, convex, equal_weight, tt_PL_cum_fx |

In [ ]:
# Helper function to check if DataFrame has required columns
def check_plot_requirements(df, plot_func_name):
    """Check if DataFrame has required columns for a plot function."""
    requirements = {
        'plot_abs_rel': ['date', 'close', 'rclose'],
        'plot_signal_bo': ['close'],  # Plus hi_N, lo_N, bo_N (dynamic)
        'plot_signal_tt': ['close'],  # Plus turtle_XY (dynamic)
        'plot_signal_ma': ['close'],  # Plus sma/ema columns (dynamic)
        'plot_signal_abs': ['close', 'hi3', 'lo3', 'clg', 'flr', 'rg_ch', 'rg'],
        'plot_signal_rel': ['rclose', 'rh3', 'rl3', 'rclg', 'rflr', 'rrg_ch', 'rrg'],
        'plot_regime_abs': ['close', 'rg', 'lo3', 'hi3', 'clg', 'flr', 'rg_ch'],
        'plot_regime_rel': ['rclose', 'rrg', 'rl3', 'rh3', 'rclg', 'rflr', 'rrg_ch'],
        'plot_profit_loss': ['tt_PL_cum', 'tt_chg1D'],
        'plot_price_signal_cumreturns': ['close', 'stop_loss', 'tt_cumul'],
        'plot_equity_risk': ['close', 'peak_eqty', 'tolerance', 'drawdown',
                             'constant_risk', 'convex_risk', 'concave_risk'],
        'plot_shares_signal': ['shs_eql', 'shs_fxd', 'shs_ccv', 'shs_cvx'],
        'plot_equity_amount': ['constant', 'concave', 'convex', 'equal_weight', 'tt_PL_cum_fx'],
    }
    
    if plot_func_name not in requirements:
        return f"Unknown function: {plot_func_name}"
    
    required = requirements[plot_func_name]
    df_cols = df.columns.tolist()
    if df.index.name:
        df_cols.append(df.index.name)
    
    missing = [col for col in required if col not in df_cols]
    
    if missing:
        return f"Missing columns: {missing}"
    return "All required columns present"

# Check our sample DataFrame
print("Column checks for sample DataFrame:")
print("=" * 50)
for func in ['plot_abs_rel', 'plot_signal_bo', 'plot_regime_abs', 
             'plot_profit_loss', 'plot_equity_amount']:
    result = check_plot_requirements(df, func)
    status = "✓" if "All required" in result else "✗"
    print(f"{status} {func}: {result}")

### Memory Management

When generating many plots (e.g., for multiple tickers), memory can accumulate. The module handles this automatically, but for custom code:

In [ ]:
# Memory-safe batch plotting
def batch_plot(df_dict, plot_func, **kwargs):
    """
    Generate plots for multiple tickers safely.
    
    Args:
        df_dict: Dict of {ticker: DataFrame}
        plot_func: Plot function to use
        **kwargs: Additional arguments for plot function
    """
    for ticker, df in df_dict.items():
        try:
            print(f"Plotting {ticker}...")
            plot_func(df, ticker, **kwargs)
            # Figures are closed automatically by the module
        except Exception as e:
            print(f"  Error for {ticker}: {e}")
            # Ensure figure is closed even on error
            plt.close('all')

# Example usage (commented out to avoid many plots)
# tickers_data = {'AAPL': df, 'GOOG': df, 'MSFT': df}
# batch_plot(tickers_data, plot_abs_rel, bm_name='SPY')
print("Batch plotting function defined.")

### Common Troubleshooting

| Issue | Cause | Solution |
|-------|-------|----------|
| `ValueError: Missing required columns` | DataFrame lacks needed columns | Check column names match expected pattern |
| `ValueError: DataFrame cannot be empty` | No data in DataFrame | Verify data loading |
| `ValueError: ticker cannot be None` | Ticker parameter is None | Provide valid ticker string |
| `KeyError: 'date'` | DataFrame already indexed | Module handles this automatically now |
| Empty plot | All values are NaN | Check data quality, fill/drop NaNs |

In [ ]:
# Quick reference card
print("""
PLOTS MODULE QUICK REFERENCE
============================

# Import
from algoshort.plots import (
    plot_abs_rel,           # Absolute vs relative price
    plot_signal_bo,         # Breakout signals
    plot_signal_tt,         # Turtle trading signals
    plot_signal_ma,         # Moving average signals
    plot_signal_abs,        # Absolute floor/ceiling
    plot_signal_rel,        # Relative floor/ceiling
    plot_regime_abs,        # Absolute regime
    plot_regime_rel,        # Relative regime
    plot_profit_loss,       # P&L visualization
    plot_price_signal_cumreturns,  # Combined view
    plot_equity_risk,       # Risk metrics
    plot_shares_signal,     # Position sizing
    plot_equity_amount,     # Equity curves
)

# Basic usage
fig, ax = plot_abs_rel(df, 'AAPL', 'SPY')

# All functions:
# - Accept DataFrame as first argument
# - Accept ticker as second argument
# - Return (Figure, Axes) tuple
# - Validate inputs automatically
# - Handle memory cleanup
""")

---

## Summary

This guide covered:

1. **Setup**: Import and configure the plots module
2. **Overview**: Available functions and common patterns
3. **Data Preparation**: Creating sample data with all required columns
4. **Price Plots**: `plot_abs_rel` for comparing absolute vs relative prices
5. **Signal Plots**: Breakout, turtle, MA, and floor/ceiling signals
6. **Regime Plots**: Visualizing regime detection results
7. **Performance Plots**: P&L, cumulative returns, and risk metrics
8. **Position Sizing**: Shares and equity curves visualization
9. **Customization**: Extending and integrating with other modules
10. **Best Practices**: Column requirements, memory management, troubleshooting

### Key Takeaways

- All functions have **consistent interfaces** (df, ticker, **kwargs)
- All functions **validate inputs** and provide helpful error messages
- All functions **prevent memory leaks** by closing figures
- All functions **return (Figure, Axes)** for potential customization
- Use the **column requirements table** to prepare your data

For questions or issues, refer to the test suite at `tests/test_plots.py`.